In [ ]:
#Installing the GPT
!pip install pytorch_pretrained_bert 


https://github.com/Apress/Natural-Language-Processing-Projects

https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1

In [2]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
%pwd

'/content'

In [10]:
import os

os.listdir('/content')

['.config', 'train.csv.zip', 'kaggle.json', 'sample_data']

In [12]:
! unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [13]:
! unzip test.csv.zip -d test

Archive:  test.csv.zip
  inflating: test/test.csv           


In [8]:
! kaggle competitions download quora-question-pairs -f train.csv.zip

 80% 17.0M/21.2M [00:00<00:00, 40.2MB/s]
100% 21.2M/21.2M [00:00<00:00, 44.8MB/s]


In [11]:
! kaggle competitions download quora-question-pairs -f test.csv.zip

 97% 168M/173M [00:03<00:00, 66.0MB/s]
100% 173M/173M [00:03<00:00, 55.7MB/s]


In [14]:
import pandas as pd
import numpy as np
import os

import scipy
import string
import csv


import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#immport tokenize, stopwords 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

#import warnings
import warnings

#import sklearn and matplotlib
from sklearn import preprocessing
import spacy
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

#import warning
warnings.filterwarnings('ignore')
import re



#Importing required tokenizer, OpenAiGPT model
import torch
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel

#import the data
train=pd.read_csv('/content/train/train.csv')
train1=train.copy()

train.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [15]:
train.shape

(404290, 6)

In [17]:
#append the both set of questions in dataset
Q1=train1.iloc[:,[2,4]]
Q2=train1.iloc[:,[1,3]]

df = pd.DataFrame( np.concatenate( (Q2.values, Q1.values), axis=0 ) )
df.columns = ['id','question' ]
df


,id,question
0,1,What is the step by step guide to invest in sh...
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,5,How can I increase the speed of my internet co...
3,7,Why am I mentally very lonely? How can I solve...
4,9,"Which one dissolve in water quikly sugar, salt..."
...,...,...
808575,379845,How many keywords are there in PERL Programmin...
808576,155606,Is it true that there is life after death?
808577,537929,What's this coin?
808578,537931,I am having little hairfall problem but I want...


In [18]:
df = df.iloc[:10000, :].copy()

In [19]:
df

,id,question
0,1,What is the step by step guide to invest in sh...
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,5,How can I increase the speed of my internet co...
3,7,Why am I mentally very lonely? How can I solve...
4,9,"Which one dissolve in water quikly sugar, salt..."
...,...,...
9995,19404,How would you order these four cities (Bangalo...
9996,19406,Stphen william hawking?
9997,19408,Mathematical Puzzles: What is () + () + () = 3...
9998,19410,Is IMS noida good for BCA?


In [ ]:
#append the both set of questions in dataset
Q1=train1.iloc[:,[2,4]]
Q2=train1.iloc[:,[1,3]]

df = pd.DataFrame( np.concatenate( (Q2.values, Q1.values), axis=0 ) )
df.columns = ['id','question' ]
df


,id,question
0,1,What is the step by step guide to invest in sh...
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,5,How can I increase the speed of my internet co...
3,7,Why am I mentally very lonely? How can I solve...
4,9,"Which one dissolve in water quikly sugar, salt..."
...,...,...
9995,9851,How does a pirate radio station work?
9996,9853,Which mobile is good within 20k?
9997,9855,What actually happened in predestination?
9998,9857,Are Near Death Experiences (NDEs) real?


# A.	 Building vectors using **Doc2Vec**

In [24]:
# importing doc2vec from gensim 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument 

# tokenizing the sentences
tok_quora=[word_tokenize(question) for question in df.question]

#creating training data
Quora_training_data=[TaggedDocument(d, [i]) for i, d in enumerate(tok_quora)]   


In [36]:
tok_quora[0]

['What',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india',
 '?']

In [25]:
# from nltk.corpus.reader.twitter import TweetTokenizer
# tweet_tokenize = TweetTokenizer(preserve_case=False, strip_handles=True)
# [tweet_tokenize.tokenize(sent) for sent in df.question]

In [29]:
Quora_training_data[0]

TaggedDocument(words=['What', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india', '?'], tags=[0])

In [26]:
# trainin doc2vec model
doc_model = Doc2Vec(Quora_training_data, vector_size = 100, window = 5, min_count = 3, epochs = 25)

In [33]:
doc_model.wv.vocab

{'What': <gensim.models.keyedvectors.Vocab at 0x7ff0613982d0>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7ff061398310>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7ff061398390>,
 'step': <gensim.models.keyedvectors.Vocab at 0x7ff0613983d0>,
 'by': <gensim.models.keyedvectors.Vocab at 0x7ff061398450>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7ff0613984d0>,
 'invest': <gensim.models.keyedvectors.Vocab at 0x7ff061398510>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7ff061398550>,
 'share': <gensim.models.keyedvectors.Vocab at 0x7ff061398410>,
 'market': <gensim.models.keyedvectors.Vocab at 0x7ff061398490>,
 'india': <gensim.models.keyedvectors.Vocab at 0x7ff061398590>,
 '?': <gensim.models.keyedvectors.Vocab at 0x7ff0613985d0>,
 'story': <gensim.models.keyedvectors.Vocab at 0x7ff061398610>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7ff061398650>,
 '(': <gensim.models.keyedvectors.Vocab at 0x7ff061398690>,
 ')': <gensim.models.keyedvectors.Vocab at 0x7ff0613986d0>,
 'How

In [34]:
#function to get vectors from model

def fetch_embeddings(model,tokens):
  tokens = [x for x in word_tokenize(tokens) if x in list(doc_model.wv.vocab)]
  #if words is not present then vector becomes zero
  if len(tokens)>=1:
    return doc_model.infer_vector(tokens)
  else:
    return np.array([0]*100)  


In [41]:
tokens = df.question[0]
tokens = [x for x in word_tokenize(tokens) if x in list(doc_model.wv.vocab)]
tokens
doc_model.infer_vector(tokens)

array([-0.03046375,  0.06748872,  0.04569051,  0.15400818, -0.13843408,
       -0.07261882, -0.08328882, -0.08133821,  0.06184241,  0.12953357,
       -0.00403365,  0.03083104, -0.12020388,  0.15500732,  0.14159241,
        0.00987293, -0.12870443, -0.03946351, -0.1487142 , -0.05289451,
       -0.00571153,  0.04004046,  0.12422021, -0.03899958,  0.06814793,
        0.15380271, -0.20053822, -0.06819471,  0.00025793,  0.10206724,
       -0.18377766, -0.00223223,  0.10381512, -0.16844963, -0.12331738,
       -0.06080144, -0.07362667, -0.13157038, -0.07719135,  0.1032192 ,
       -0.15635072, -0.06107268,  0.17315538, -0.03912927, -0.03336195,
       -0.07159454, -0.0824202 ,  0.12881634,  0.04915957,  0.09614407,
       -0.03562962, -0.12349634, -0.0090621 , -0.07893188,  0.11887623,
        0.16429459,  0.15984361,  0.06805628, -0.03648902, -0.11801489,
        0.04080083, -0.15206653, -0.06410917,  0.09033136,  0.08982357,
        0.2333689 ,  0.01699019,  0.06519176, -0.15230028,  0.03

In [42]:
#Storing all embedded sentence vectors in a list 

#defining empty list and iterating through all the questions

doc_embeddings=[]                                     
for w in df.question:
    doc_embeddings.append(list(fetch_embeddings(doc_model, w)))  
#conveting it into array
doc_embeddings=np.asarray(doc_embeddings)


In [44]:
#Since there are 10,000 documents in the training set, we should see 10,000 document embeddings
len(doc_embeddings)

10000

In [46]:
#Since we set the vector_size to 100, Doc2Vec(Quora_training_data, vector_size = 100, window = 5, min_count = 3, epochs = 25)
len(doc_embeddings[0])

100

# B.	 Sentence Transformers using BERT model

In [47]:
#install SBERT
!pip install sentence-transformers

#import the SBERT
from sentence_transformers import SentenceTransformer

#let use paraphrase-MiniLM-L12-v2 pre trained model
sbert_model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

x=[i for i in df.question]
#lets get embeddings for each question
sentence_embeddings_BERT= sbert_model.encode(x)

#lets see the shape
sentence_embeddings_BERT.shape

sentence_embeddings_BERT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 5.5 MB 8.3 MB/s 
     |████████████████████████████████| 1.3 MB 8.5 MB/s 
     |████████████████████████████████| 182 kB 5.6 MB/s 
     |████████████████████████████████| 7.6 MB 51.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ceb52594f51496a4063fce0de1c0bdfc6152d3a771c36b26af4a75fc4e6aa188
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

array([[-0.15299675, -0.30485192,  0.00183824, ..., -0.27034354,
        -0.42608353,  0.31928638],
       [-0.16776407,  0.67119473, -0.51778895, ..., -0.08420178,
         0.004704  ,  0.32004666],
       [ 0.09468901, -0.00629804,  0.06894324, ..., -0.18716967,
        -0.2556717 ,  0.04188575],
       ...,
       [-0.04815659,  0.10775804, -0.07293894, ...,  0.12091764,
         0.23045003,  0.16716625],
       [-0.05122644, -0.24057986,  0.08293396, ..., -0.6683932 ,
        -0.0127978 ,  0.18347953],
       [-0.24609756,  0.29296714, -0.04927419, ...,  0.09974647,
         0.03092924,  0.12593076]], dtype=float32)

# C.	GPT

In [48]:

#initializing the tokenizer
tok_gpt= OpenAIGPTTokenizer.from_pretrained('openai-gpt')  

#Initializing the gpt Model
model_gpt= OpenAIGPTModel.from_pretrained('openai-gpt')
model_gpt.eval()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 27.3 MB/s 
     |████████████████████████████████| 132 kB 57.3 MB/s 
     |████████████████████████████████| 10.2 MB 48.0 MB/s 
     |████████████████████████████████| 79 kB 6.1 MB/s 
     |████████████████████████████████| 140 kB 19.3 MB/s 
     |████████████████████████████████| 127 kB 66.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


ImportError: ignored

In [49]:
def Fetch_gpt_vectors(question):
  
  #tokenize words
  words = word_tokenize(question)
  emb = np.zeros((1,768))

  #get vectore for each word
  for word in words:
      w= tok_gpt.tokenize(word)
      indexed_words = tok_gpt.convert_tokens_to_ids(w)
      tns_word = torch.tensor([indexed_words])

      with torch.no_grad():
          try:
     #get mean vector
            emb += np.array(torch.mean(model_gpt(tns_word),1))
          except Exception as e:
            continue
  
  emb /= len(words)
  return emb

gpt_emb = np.zeros((1000, 768))

# get vectors

for v in range(1000):
    txt = df.loc[v,'question']
    
    gpt_emb[v] = Fetch_gpt_vectors(txt)

gpt_emb


NameError: ignored

# Finding Similar questions

In [62]:
#defining function to derive cosine similarity

#import 
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

# def cosine_similarity(vec1,vec2):
#     #find the score
#     return dot(vec1, vec2)/(norm(vec1)*norm(vec2)) 


In [64]:
vec1 = get_input_vector('How is Narendra Modi as a person?','Doc2Vec')
vec2 = get_input_vector('Is Narendra Modi president of India?','Doc2Vec')


How is Narendra Modi as a person?
Is Narendra Modi president of India?


(1, 100)

In [85]:
x = cosine_similarity(vec1, doc_embeddings)[0]
temp_list=list(x)
#sorting
sort_res = sorted(range(len(x)), key = lambda sub: x[sub])[:]
newsort_res = np.argsort(x)
sim_score=[temp_list[i] for i in reversed(sort_res)]
#print
print([idx for idx in reversed(sort_res[0:5])])
print([idx for idx in reversed(newsort_res[0:5])])

[6060, 3512, 3508, 8367, 4496]
[6060, 3512, 3508, 8367, 4496]


In [73]:
 from sklearn.metrics.pairwise import cosine_similarity
    #getting cosine similarities of overall data set with input queries from user
    x=cosine_similarity(user,embeddings).tolist()[0]
    
    #sorting
    #temp_list=list(x)
    #sort_res = sorted(range(len(x)), key = lambda sub: x[sub])[:]
    sort_res = np.argsort(x)
    #sim_score=[temp_list[i] for i in reversed(sort_res)]

    #print
    print(sort_res[0:5])
 
    #index fetching
    L=[]
    for i in reversed(sort_res):
        L.append(i)

array([[0.01806104]], dtype=float32)

In [65]:
cosine_similarity(vec1,vec2)


array([[0.45641124]], dtype=float32)

In [86]:
#Function which gts Top N similar questions from data 

def top_n_questions(user,embeddings,df):    
    from sklearn.metrics.pairwise import cosine_similarity
    #getting cosine similarities of overall data set with input queries from user
    x=cosine_similarity(user,embeddings).tolist()[0]
    #temp_list=list(x)

    #sorting
    #sort_res = sorted(range(len(x)), key = lambda sub: x[sub])[:]
    #sim_score=[temp_list[i] for i in reversed(sort_res)]

    #print
    sort_res = np.argsort(x)
    print(sort_res[0:5])
 
    #index fetching
    L=[]
    for i in reversed(sort_res):
        L.append(i)

    #get the index from dataframe 
    return df.iloc[L[0:5], [0,1]]

#function to fetch the results based on the model selected

def get_input_vector(query,model):    
    
    print(query)

    #Doc2vec model
    if model=='Doc2Vec':
      k=fetch_embeddings(doc_model,query)
      k=k.reshape(1, -1)

    # sbert  model
    elif model=='BERT':
      k=sbert_model.encode(str(query))
      k=k.reshape(1, -1)

    # gpt model
    elif model=='GPT':
      k=Fetch_gpt_vectors(query)  

    return k


In [98]:
pd.options.display.max_colwidth=300
df[df['id'] == 7]

#cosine_similarity(get_input_vector('Why am I mentally very lonely?', 'Doc2Vec'),get_input_vector("Why am I mentally very lonely? How can I solve it?", 'Doc2Vec'))
#[0.74149984]

cosine_similarity(get_input_vector('Why am I mentally very lonely?', 'Doc2Vec'),get_input_vector("Why am I afraid of pain?", 'Doc2Vec'))
#[[0.7581431]]

cosine_similarity(get_input_vector('Why am I mentally very lonely?', 'Doc2Vec'),get_input_vector("Why am I angry with myself?", 'Doc2Vec'))
#[[0.7648239]]


Why am I mentally very lonely?
Why am I afraid of pain?
Why am I mentally very lonely?
Why am I angry with myself?


array([[0.7648239]], dtype=float32)

In [91]:
# Example 1 - Doc2vec model

top_n_questions(get_input_vector('Why am I mentally very lonely?', 'Doc2Vec'),doc_embeddings,df) 


Why am I mentally very lonely?
[3856 8478 5365 1730 4136]


,id,question
8241,16074,Why am I afraid of pain?
6351,12447,Why am I angry with myself?
3,7,Why am I mentally very lonely? How can I solve...
7358,14377,Why am I feeling nauseous and have diarrhea?
6251,12253,Why am I so alone and have no friends?


In [88]:
# Example 2 - GPT model


top_n_questions(get_input_vector('How is Narendra Modi as a person?','GPT'),gpt_emb,df) 


How is Narendra Modi as a person?


NameError: ignored

In [89]:
# Example 3 - BERT

top_n_questions(get_input_vector('How is Narendra Modi as a person?','BERT'),sentence_embeddings_BERT,df) 


How is Narendra Modi as a person?
[9398 6009 4873 2412 8246]


,id,question
2166,4310,How is Narendra Modi as a person?
5998,11766,How can I meet Narendra Modi?
3628,2267,How can I meet Modi?
1137,2266,How can I get a chance to meet Mr. Narendra Modi?
1826,3635,What is the diet of Narendra Modi?


# **Implementation: Supervised Learning**

In [121]:
# import packages required.
import pandas as pd
import numpy as np
import scipy
import os
import string
import csv

#import nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#import tokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 

#import warnings
import warnings

#import sklearn and matplotlib
from sklearn import preprocessing
import spacy
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

#import warning
warnings.filterwarnings('ignore')
import re

from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

#import Tokenizer from keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

#importing Keras necessary libraries
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Dropout, LSTM


#importing train data - Import the full data
quora_questions=pd.read_csv('/content/train/train.csv') 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [122]:
quora_questions.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0


In [124]:
quora_questions = quora_questions.iloc[:10000, :]
quora_questions

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
9995,9995,19404,19405,"How would you order these four cities (Bangalore, Chennai, Hyderabad, Pune) for living (considering climate, people, food, future growth, safety, cost of living, etc.)?",What is the cost of living in Europe and the US compared with Indian cities like Chennai and Bangalore?,0
9996,9996,19406,19407,Stphen william hawking?,"What are the differences between SM, YG and JYP auditions? Are the other entertainment companies also different in their auditioning system?",0
9997,9997,19408,19409,"Mathematical Puzzles: What is () + () + () = 30 using 1,3,5,7,9,11,13,15?",What are the steps to solve this equation: [math]\frac{x}{x-5} + \frac{3}{2x-10} - \frac{1-x}{6x-30} + \frac{7}{15-3x} =1[/math]?,0
9998,9998,19410,19411,Is IMS noida good for BCA?,How good is IMS Noida for studying BCA?,1


In [125]:
#function for data cleaning
def txt_process(input_text):

    #print(input_text)
    # Removing punctuation from input text
    input_text = ''.join([x for x in input_text if x not in punctuation])
    
    # Cleaning the text
    input_text = re.sub(r"[^A-Za-z0-9]", " ", input_text)
    input_text = re.sub(r"\'s", " ", input_text)
      
    # remove stop words
    input_text = input_text.split()
    input_text = [x for x in input_text if not x in stop_words]
    input_text = " ".join(input_text)
    
    # Return a list of words
    return(input_text)




In [126]:

#applying above function on both question ids
quora_questions['question1_cleaned'] = quora_questions.apply(lambda x: txt_process(x['question1']), axis = 1)  
quora_questions['question2_cleaned'] = quora_questions.apply(lambda x: txt_process(x['question2']), axis = 1)

quora_questions.head(10)

,id,qid1,qid2,question1,question2,is_duplicate,question1_cleaned,question2_cleaned
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0,What step step guide invest share market india,What step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0,What story Kohinoor KohiNoor Diamond,What would happen Indian government stole Kohinoor KohiNoor diamond back
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0,How I increase speed internet connection using VPN,How Internet speed increased hacking DNS
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0,Why I mentally lonely How I solve,Find remainder math2324math divided 2423
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0,Which one dissolve water quikly sugar salt methane carbon di oxide,Which fish would survive salt water
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?,"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",1,Astrology I Capricorn Sun Cap moon cap risingwhat say,Im triple Capricorn Sun Moon ascendant Capricorn What say
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone and video games?,0,Should I buy tiago,What keeps childern active far phone video games
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,How I good geologist,What I great geologist
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,When use instead,When use instead
9,9,19,20,Motorola (company): Can I hack my Charter Motorolla DCX3400?,How do I hack Motorola DCX3400 for free internet?,0,Motorola company Can I hack Charter Motorolla DCX3400,How I hack Motorola DCX3400 free internet


In [127]:
#stacking
question_text = np.hstack([quora_questions.question1_cleaned, quora_questions.question2_cleaned])

#tokenizing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(question_text)

#creating new columns for both ids where tokenized form of sentence is created 
quora_questions['tokenizer_1'] = tokenizer.texts_to_sequences(quora_questions.question1_cleaned)
quora_questions['tokenizer_2'] = tokenizer.texts_to_sequences(quora_questions.question2_cleaned)

quora_questions.head(5)


,id,qid1,qid2,question1,question2,is_duplicate,question1_cleaned,question2_cleaned,tokenizer_1,tokenizer_2
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0,What step step guide invest share market india,What step step guide invest share market,"[1, 1677, 1677, 2358, 634, 497, 363, 10]","[1, 1677, 1677, 2358, 634, 497, 363]"
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0,What story Kohinoor KohiNoor Diamond,What would happen Indian government stole Kohinoor KohiNoor diamond back,"[1, 352, 3869, 3869, 3870]","[1, 14, 126, 36, 167, 6528, 3869, 3869, 3870, 121]"
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0,How I increase speed internet connection using VPN,How Internet speed increased hacking DNS,"[3, 2, 127, 421, 259, 1678, 101, 2944]","[3, 259, 421, 2406, 1009, 12681]"
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0,Why I mentally lonely How I solve,Find remainder math2324math divided 2423,"[4, 2, 2600, 2601, 3, 2, 683]","[41, 4644, 12682, 5872, 12683]"
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0,Which one dissolve water quikly sugar salt methane carbon di oxide,Which fish would survive salt water,"[8, 15, 3871, 159, 8968, 1369, 1679, 4685, 1463, 5940, 8969]","[8, 3069, 14, 1218, 1679, 159]"


In [128]:
#combining both tokens in one list question1 followed by question2
quora_questions['tokenizer'] = quora_questions['tokenizer_1'] + quora_questions['tokenizer_2']

#defining max length 
m_len = 500

#max tokens
max_token = np.max(quora_questions.tokenizer.max())


In [132]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#defining X and target data 
y = quora_questions[['is_duplicate']]
X = quora_questions[['tokenizer']]

#padding X with a maximum length
X = pad_sequences(X.tokenizer, maxlen = m_len)

#splitting data into train and test
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=10)


#defining the LSTM model
quora_model = Sequential()

#adding embeedding layer
quora_model.add(Embedding(70000, 64))

#adding drop out layer
quora_model.add(Dropout(0.15))

#LSTM layer
quora_model.add(LSTM(16))

#adding sigmoid layer
quora_model.add(Dense(1, activation = 'sigmoid'))

#defining loss and optimizer
quora_model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])


quora_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          4480000   
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 lstm (LSTM)                 (None, 16)                5184      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 4,485,201
Trainable params: 4,485,201
Non-trainable params: 0
_________________________________________________________________


In [133]:
#training the model and validating on test data
quora_model.fit(X_train, y_train, epochs = 2, batch_size=64,validation_data=(X_test,y_test))


Epoch 1/2
118/118 [==============================] - 24s 182ms/step - loss: 0.6763 - accuracy: 0.6283 - val_loss: 0.6681 - val_accuracy: 0.6248
Epoch 2/2
118/118 [==============================] - 22s 183ms/step - loss: 0.6629 - accuracy: 0.6303 - val_loss: 0.6630 - val_accuracy: 0.6248


In [ ]:
# evaluation of the model
import sklearn
from sklearn.metrics import classification_report

#prediction on train data
tr_prediction=quora_model.predict(X_train) 

#replacing probabilities >0.5 with 1 and other 0
tr_prediction[tr_prediction>0.5]=1 
tr_prediction[tr_prediction<0.5]=0
tr_prediction

#true values of train data
tr_true=y_train.values

#accuracy
Accuracy=sklearn.metrics.accuracy_score(np.array(tr_true), 
                                     np.array(tr_prediction))

print(Accuracy)
0.7811906400332337

#classification report with f1 score

print(classification_report(tr_true, tr_prediction, target_names=['Not similar','similar']))


0.6165333333333334
              precision    recall  f1-score   support

 Not similar       0.62      1.00      0.76      2312
     similar       0.00      0.00      0.00      1438

    accuracy                           0.62      3750
   macro avg       0.31      0.50      0.38      3750
weighted avg       0.38      0.62      0.47      3750



In [136]:
import sklearn
from sklearn.metrics import classification_report
#predicting on test data
test_prediction=quora_model.predict(X_test)

#generating classes
test_prediction[test_prediction>0.5]=1 
test_prediction[test_prediction<0.5]=0
test_prediction

#true values for test
test_true=y_test.values

# accuracy on test data
Accuracy=sklearn.metrics.accuracy_score(np.array(test_true), 
                                     np.array(test_prediction))

print('Accuracy is %f'%(Accuracy*100)+' %')

print(classification_report(test_true, test_prediction, target_names=['Not similar','similar']))


79/79 [==============================] - 3s 39ms/step
Accuracy is 62.480000 %
              precision    recall  f1-score   support

 Not similar       0.62      1.00      0.77      1562
     similar       0.00      0.00      0.00       938

    accuracy                           0.62      2500
   macro avg       0.31      0.50      0.38      2500
weighted avg       0.39      0.62      0.48      2500



In [139]:
def find_similarity_score(q1,q2):

  #clean first question
  Q1_C=  txt_process(q1) 
  #print(q1)

  #clean first question
  Q2_C = txt_process(q2)    
  #print(q2)

  #converting 1st question into tokens
  Q1_C = tokenizer.texts_to_sequences([Q1_C])  

  #converting 2nd question into token
  Q2_C = tokenizer.texts_to_sequences([Q2_C])

  #combining both tokens as we did for train data
  Q_final = Q1_C[0] + Q2_C[0]                      
  
  #padding combined sequence to max length
  Q_Test = pad_sequences([Q_final], maxlen = 500)  
  
  #predicting probability of given pair
  Prob=quora_model.predict(Q_Test)         
  print(Prob)

  #if p>0.5 then similar
  if Prob[0]>0.5:                  
    return 'Quora Questions are similar'
  else:
    return 'Quora Questions are Not similar'


In [140]:
#example 1
find_similarity_score('Who is Narendra Modi?','What is identity of Narendra Modi?')

1/1 [==============================] - 0s 48ms/step
[[0.39593637]]


'Quora Questions are Not similar'

In [141]:
#example 2
find_similarity_score('is there life after death?','Do people belive in afterlife')


1/1 [==============================] - 0s 25ms/step
[[0.3888578]]


'Quora Questions are Not similar'

In [142]:
#example 3
find_similarity_score('Should I have a hair transplant at age 24? How much would it cost?','How much cost does hair transplant require?')


1/1 [==============================] - 0s 30ms/step
[[0.3982434]]


'Quora Questions are Not similar'